In [89]:
import numpy as np
import copy
import time

In [90]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

assert pyro.__version__.startswith('1.8.4')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

In [91]:
def generate_stdp_dataset(dim, num_examples, min_value, max_value):
    X = np.random.random((num_examples + 1, dim)) * (max_value - min_value) + min_value
    beta = np.random.random((dim)) * (max_value - min_value) + min_value

    noise = np.random.normal(0, np.sqrt(max_value - min_value), num_examples + 1)
    Y = X[:num_examples + 1] @ beta + noise

    X = np.asfortranarray(X)
    Y = np.asfortranarray(Y)
    X /= np.linalg.norm(X, axis=0)
    Y = (Y - Y.mean()) / Y.std()
    Y = Y * max_value

    Y = Y/np.linalg.norm(Y)

    return X, Y, beta

In [92]:
std0 = torch.eye(3, dtype=torch.float64) * 0.3
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(3, dtype=torch.float64)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.MultivariateNormal(mu0, std0))
    # loop over the observed data
    for i in range(len(data)):
        pyro.sample("obs_{}".format(i), dist.Normal(f.dot(data[i][0]), 0.3), obs=data[i][1])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    mu_q = pyro.param("mu_q", torch.zeros(3, dtype=torch.float64))
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.MultivariateNormal(mu_q, std0))

In [93]:
X, Y, beta = generate_stdp_dataset(3, 10, 0, 1)

In [94]:
print(X)
print(Y)
print(beta)

[[0.29999668 0.48202011 0.13952446]
 [0.21416448 0.1359368  0.39056766]
 [0.10713974 0.15606481 0.35426628]
 [0.2616496  0.1559739  0.41479371]
 [0.0648895  0.08377738 0.27131187]
 [0.32908907 0.48322414 0.28626144]
 [0.3951819  0.56568504 0.17928333]
 [0.48373391 0.15520244 0.32141237]
 [0.18946765 0.33999251 0.36192091]
 [0.41851802 0.00630068 0.29390687]
 [0.26544926 0.0157334  0.14852353]]
[-0.02372209 -0.09527483 -0.07850806 -0.26222199 -0.28382719  0.21530555
  0.44547127  0.60387861 -0.06534259 -0.47003637  0.0142777 ]
[0.5093743  0.47112557 0.72330546]


In [95]:
X_train = copy.deepcopy(X)
Y_train = copy.deepcopy(Y[:len(Y) - 1])
X_train = [torch.tensor(member) for member in X_train]
Y_train = [torch.tensor(member) for member in Y_train]

In [96]:
np.std(Y_train)

0.3161923088767153

In [97]:
def train_SVI(D_hat, n_steps):
    # setup the optimizer
    adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # do gradient steps
    for step in range(n_steps):
        svi.step(D_hat)
    
    breakpoint()
    mu_q = pyro.param("mu_q")
    return mu_q

In [98]:
z = [0 for i in range(len(Y_train) + 1)]
print(z)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [99]:
y_hat = max(Y_train)
y_bottom = min(Y_train)
print(y_hat)
print(y_bottom)
conformal_set = []
step_size = 0.1
start = time.time()
while y_hat >= y_bottom:
    pyro.clear_param_store()
    # Create D_hat
    D_hat = list(zip(X_train[:-1], Y_train))
    D_hat.append((X_train[-1], y_hat))
    
    # Train SVI
    mu_q = train_SVI(D_hat, 10)
    
    # Calculate rank of y_hat
    rank = [(sum(D_hat[i][0] * mu_q) - D_hat[i][1]).detach().numpy() for i in range(len(D_hat))]
    y_hat_rank = rank[-1]
    
    # Add to conformal set if in not in bottom 10 percent of probabilities
    if np.count_nonzero(y_hat_rank > rank) / len(rank) > 0.1:
        conformal_set.append(copy.deepcopy(y_hat))
    else:
        print(f"{y_hat} Not added")
        
    y_hat -= step_size
conformal_set = [min(conformal_set), max(conformal_set)]
end = time.time()

tensor(0.6039, dtype=torch.float64)
tensor(-0.4700, dtype=torch.float64)
0.6038786081870835 Not added
0.5038786081870835 Not added
0.40387860818708354 Not added
0.30387860818708357 Not added


In [100]:
print(conformal_set)
print(f"Y[-1]: {Y[-1]}")
if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
    print(f"Y[-1] is covered")
else:
    print("Y[-1] is Not covered")
print(f"Elapsed Time: {end - start}")

[tensor(-0.3961, dtype=torch.float64), tensor(0.2039, dtype=torch.float64)]
Y[-1]: 0.014277702645567266
Y[-1] is covered
Elapsed Time: 0.5369627475738525
